<a href="https://colab.research.google.com/github/teticio/audio-diffusion/blob/master/notebooks/train_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
try:
    # are we running on Google Colab?
    import google.colab
    !git clone -q https://github.com/teticio/audio-diffusion.git
    %cd audio-diffusion
    %pip install -q -r requirements.txt .
except:
    pass

In [ ]:
import sys
from IPython.display import Audio
from audiodiffusion import AudioDiffusion

### Upload / specify audio files to train on
Provide some MP3 or WAV files that will be split into samples and converted to Mel spectrograms. For a resolution of 256, the samples will be about 5 seconds long.

In [ ]:
try:
    # are we running on Google Colab?
    from google.colab import files
    input_dir = '.'
    files.upload();
except:
    input_dir = "."

### Prepare dataset

In [ ]:
!{sys.executable} scripts/audio_to_images.py \
  --resolution 256,256 \
  --input_dir {input_dir} \
  --output_dir data

### Train model
The DDIM scheduler generates samples much faster.

In [ ]:
!{sys.executable} scripts/train_unet.py \
  --dataset_name data \
  --output_dir models/model \
  --num_epochs 10 \
  --train_batch_size 2 \
  --eval_batch_size 2 \
  --gradient_accumulation_steps 8 \
  --save_images_epochs 100 \
  --save_model_epochs 1 \
  --scheduler ddim

### Generate samples with model

In [ ]:
audio_diffusion = AudioDiffusion("models/model")

In [ ]:
image, (sample_rate, audio) = audio_diffusion.generate_spectrogram_and_audio()
display(image)
display(Audio(audio, rate=sample_rate))